<a href="https://colab.research.google.com/github/dlskawns/cp1/blob/main/Keyword_extraction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 키워드 추출 모델

## Sentence Transformer - Bert(distilbert-base-nli-mean-tokens)를 이용 

* 선정 이유:  
NLI(Natural Language Inference) 작업에 적합한 모델로, 문장(doc)단위 임베딩이 가능하다. 

* 가설:  
문장단위 임베딩을 진행 후, 단어 임베딩의 유사도를 파악해 해당 문장 내 중요 키워드를 찾아낼 수 있다.

* 방법:  
  * 빈도 수 기반 벡터화를 진행 (Counter Vectorizer) - n gram(3,3)으로 단어 묶음 설정
  * sentence transformer로 샘플 별 문장 임베딩 생성
  * sentence transformer로 샘플 별 단어 임베딩 생성
  * Cosine 유사도를 통해 특정 중요 단어 파악
  * 동일한 방법으로 n gram(1,1)로 진행하여 최종 키워드 top 5~8개 추출




In [ ]:
# 트랜스포머 모델 생성
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### 추출모델 작성

In [ ]:
top_n = 10            # 빈도 수 높은 단어 상위 10개 추출
k_word, emb =[], []

for i in range(len(df['Text'])):
  print(i)
  n_gram_range = (3, 3)
  # Extract candidate words/phrases
  try:  
      count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([df['Text'][i]])

      candidates = count.get_feature_names()
      doc_embedding = model.encode([df['Text'][i]])
      candidate_embeddings = model.encode(candidates)
      distances = cosine_similarity(doc_embedding, candidate_embeddings)
      keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

      n_gram_range1 = (1, 1)
      count1 = CountVectorizer(ngram_range=n_gram_range1, stop_words=stop_words).fit(keywords)
      candidates1 = count1.get_feature_names()

      doc_embedding1 = model.encode(keywords)
      candidate_embeddings1 = model.encode(candidates1)
      distances1 = cosine_similarity(doc_embedding1, candidate_embeddings1)
      a = [candidates1[index] for index in distances1.argsort()[0][-5:] if pos_tag(candidates1[index])[0][1] == 'JJ' if pos_tag(candidates1[index])[0][1] == 'NN' or pos_tag(candidates1[index])[0][1] == 'NNS']
      b = [index for index in distances1.argsort()[0][-5:] if pos_tag(candidates1[index])[0][1] == 'JJ' if pos_tag(candidates1[index])[0][1] == 'NN' or pos_tag(candidates1[index])[0][1] == 'NNS']
      if len(a) < 3:
        a = [candidates1[index] for index in distances1.argsort()[0][-5:]]
        
      k_word.append(a)
      emb.append(b)
  except ValueError:
    try:
      count = CountVectorizer(ngram_range=n_gram_range, stop_words='english').fit([df['Text'][i]])
      candidates = count.get_feature_names()
      doc_embedding = model.encode([df['Text'][i]])
      candidate_embeddings = model.encode(candidates)
      distances = cosine_similarity(doc_embedding, candidate_embeddings)
      keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

      n_gram_range1 = (1, 1)
      count1 = CountVectorizer(ngram_range=n_gram_range1, stop_words='english').fit(keywords)
      candidates1 = count1.get_feature_names()

      doc_embedding1 = model.encode(keywords)
      candidate_embeddings1 = model.encode(candidates1)
      distances1 = cosine_similarity(doc_embedding1, candidate_embeddings1)
      a = [candidates1[index] for index in distances1.argsort()[0][-5:] if pos_tag(candidates1[index])[0][1] == 'NN' or pos_tag(candidates1[index])[0][1] == 'NNS' or pos_tag(candidates1[index])[0][1] == 'JJ']
      if len(a) < 3:
        a = [candidates1[index] for index in distances1.argsort()[0][-5:]]
        b = [index for index in distances1.argsort()[0][-5:]]
      k_word.append(a)
      emb.append(b)
    except ValueError:
      k_word.append('No')
      emb.append('')
df['keyword']= k_word
df['emb']= emb

In [ ]:
df[['Text','keyword']][:10]

,Text,keyword
0,I have bought several of the Vitality canned d...,"['vitality', 'finicky', 'smells', 'labrador', ..."
1,Product arrived labeled as Jumbo Salted Peanut...,"['jumbo', 'unsalted', 'small', 'sized', 'vendo..."
2,This is a confection that has been around a fe...,"['gelatin', 'pillowy', 'mouthful', 'coated', '..."
3,If you are looking for the secret ingredient i...,"['medicinal', 'robitussin', 'cherry', 'ingredi..."
4,Great taffy at a great price. There was a wid...,"['quick', 'wide', 'delivery', 'deal', 'lover']"
5,I got a wild hair for taffy and ordered this f...,"['treat', 'beer', 'melon', 'peppermint', 'poun..."
6,This saltwater taffy had great flavors and was...,"['version', 'stuck', 'themed', 'fralinger', 'c..."
7,This taffy is so good. It is very soft and ch...,"['taffy', 'soft', 'satisfying', 'amazing', 'ch..."
8,Right now I'm mostly just sprouting this so my...,"['mostly', 'sprouting', 'rye', 'grass', 'wheat..."
9,This is a very healthy dog food. Good for thei...,"['digestion', 'healthy', 'feeding', 'small', '..."


* 키워드 추출 모델 결과:  
  * 형용사 키워드 추출: smells, unsalted, chewy 등  
  * 명사 키워드 추출: stew, gelatin, delivery, digestion, meat 등
  * 해당 키워드와 함께 된 scoring으로 상품 평 분석 가능

* 문제 및 개선점:
  * 리뷰 내용이 매우 적거나 불용어만으로 이뤄진 경우 키워드가 나오지 않을 수 있음. 적은 수로 존재하기 때문에, No 라는 키워드로 반환
  * 불용어가 100% 처리되지 않아 mostly 등과 같은 단어가 여전히 존재하므로 더 정교한 불용어 처리 필요